# Permit Tools

These tools integrate [Permit.io](https://www.permit.io/) with LangChain to handle two critical security needs:

1. **JWT Validation Tool** (`LangchainJWTValidationTool`):  
   Validates a JWT token by fetching the correct public key from a JWKS endpoint and returning the token’s claims.

2. **Authorization Tool** (`LangchainPermitTool`):  
   Checks whether a user (identified by ID) has permission to perform a specific action on a resource in Permit.

By combining both tools, you can securely **authenticate** and **authorize** users or agents within LangChain workflows.

---

## Overview

### What You Get

- **Authentication via JWT**  
  Verifies token signatures against public JWKs, extracts user claims, and ensures only valid tokens proceed.

- **Fine-Grained Authorization**  
  Uses Permit.io’s API to check if a given user is allowed to perform an action on a resource, enforcing permissions within your LangChain flows.

### Tool Features

| Tool Name                   | Purpose                                             | Returns                  |
|-----------------------------|-----------------------------------------------------|--------------------------|
| **LangchainJWTValidationTool** | Validates JWT tokens via JWKS and returns claims    | `dict` (decoded claims)  |
| **LangchainPermitTool**       | Checks a user’s permission (via `Permit`)          | `bool` (allowed or not)  |

---

## Setup

1. **Install the Integration**  
   ```bash
   pip install langchain-permit

2. Environment Variables

- **PERMIT_API_KEY** – Required for the Permit authorization tool.  
- *(Optional)* Provide a **JWKS_URL** or pass one at runtime for JWT validation if you like.

For more details on setting up a Permit project or obtaining your `PERMIT_API_KEY`, see [Permit.io Documentation](https://docs.permit.io/).

## Credentials

Depending on which tool you use, you may need additional credentials or configuration:

### For LangchainJWTValidationTool

- A **JWKS endpoint** that publishes your public keys in JSON Web Key Set format.  
- The tool dynamically fetches the public key by matching the `kid` in the JWT’s header.

### For LangchainPermitTool

- **PERMIT_API_KEY** must be set in your environment or passed in as part of the tool’s initialization.  
- Permit’s dashboard defines which actions and resources are allowed for each user, role, or group.

## Instantiation

Below is how to instantiate each tool in Python. Typically, you’ll import them from the `langchain_permit.tools` module:

```python
from langchain_permit.tools import LangchainJWTValidationTool, LangchainPermitTool

# Initialize JWT validation tool (default approach)
jwt_tool = LangchainJWTValidationTool()

# Initialize Permit-based authorization tool
permit_tool = LangchainPermitTool()

By default, these will look for environment variables (PERMIT_API_KEY, etc.). However, you can also pass parameters (like a custom Permit client or specific JWKS URL) to the constructor if needed.

## Invocation

### Invoke Directly with Args

You can call each tool’s `.run()` method (sync) or `.arun()` method (async) directly:

```python
# 1. Validate JWT
jwt_token = "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9..."
claims = jwt_tool.run(jwt_token=jwt_token, jwks_url="https://example.com/.well-known/jwks.json")
print("Decoded Claims:", claims)

# 2. Check Permit-based Authorization
user_id = claims.get("sub")  # or any claim storing the user ID
action = "read"
resource = "document"

allowed = permit_tool.run(user=user_id, action=action, resource=resource)
print("Is user allowed?", allowed)
```
If allowed is True, the user can proceed. Otherwise, you can deny the request or take another action.
